In [ ]:
from pathlib import Path
import json
import h5py

# Simulation of signal propagation on a hexagonal lattice by lateral induction

This notebook is intended to help the user to reproduce simulation results for the following manuscript:

    Control of spatio-temporal patterning via cell density in a multicellular synthetic gene circuit
    Marco Santorelli, Pranav S. Bhamidipati, Andriu Kavanagh, Victoria A. MacKrell, Trusha Sondkar, Matt Thomson, Leonardo Morsut
    bioRxiv 2022.10.04.510900; doi: https://doi.org/10.1101/2022.10.04.510900
    
Before beginning this tutorial, please follow the installation instructions in `README.md`, including the creation of an IPython kernel for this notebook. All the contents for this package are contained in the folder `lateral_signaling`. Please note that all scripts in the package are intended to be run from inside this folder!

---

The main module contents are in `lateral_signaling.py`, so we import that first.

In [ ]:
import lateral_signaling as lsig

The module contains simulations of a few different scenarios. We will first see how it works in the default case ("`basicsim`") with a single sender cell in the midst of a hexagonal lattice of transceivers, with cell density dynamics taken into account. To keep track of all metadata for our simulations such as parameter configuration, system configuration, random seeds, etc. we use the data provenance package `sacred`. 

__Running a simulation script__

To run a working example, you can simply execute one of the simulation scripts at the command line (e.g. `python simulate_basicsim_run_increasingdensity.py`).

__A simulation example__

A more detailed walkthrough of what happens when you run a script such as the one above. The first step is to import this simulation ("experiment" in `sacred`'s parlance) from the corresponding `run_one` script.

In [ ]:
from simulate_basicsim_run_one import ex

We can inspect this object to see its metadata.

In [ ]:
from pprint import pprint

print("Experiment info:\n")
pprint(ex.get_experiment_info())

print(f"\nDirectory for saving results:\n\t{ex.observers[0].basedir}\n")

This information, along with more information stored at experiment run-time, enables reproduction of results. 

To run this experiment with the default configuration, simply execute the `run` method with no arguments.

In [ ]:
first_run = ex.run()

We can inspect the metadata and outputs of this run, which are saved in a numbered subfolder of the `sacred` directory.

In [ ]:
run_dir = Path(first_run.observers[0].dir)
print("\nResults and metadata were saved here : ", run_dir.absolute())
print("\nFiles created:", *[d.name for d in run_dir.glob("*")], sep="\n\t")

The `run.json` file contains metadata for this run, and `config.json` contains the default parameter configuration, along with the random number generator seed used for this run.

In [ ]:
print("config.json:\n")
pprint(json.load(run_dir.joinpath("config.json").open("r")))

`results.hdf5` contains the saved output of the simulation in HDF format. Let's inspect this to see what was saved.

In [ ]:
import numpy as np

with h5py.File(run_dir.joinpath("results.hdf5"), "r") as f:
    pprint(f.keys())

This run saved the simulation time-points `t`, density over time `rho_t`, expression over time for reporter `R_t` and signaling ligand `S_t`, the index of the single sender cell `sender_idx`, and the xy coordinates of cells in the hexagonal lattice `X`.

Let's extract time and expression information and plot example expression for the sender cell and one of its neighbors.

In [ ]:
with h5py.File(run_dir.joinpath("results.hdf5"), "r") as f:
    
    t = np.array(f["t"])
    t_days = lsig.t_to_units(t)
    
    rho_t = np.array(f["rho_t"])
    S_t = np.array(f["S_t"])
    R_t = np.array(f["R_t"])
    sender_idx = np.array(f["sender_idx"])
    
    X = np.array(f["X"])                    # Stores cell coordinates for rho=1
    X = X - X[sender_idx]                   # Center coordinates on the sender cell
    X_t = lsig.transform_lattice(X, rho_t)  # Get coordinates over time as density increases

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(t, S_t[:, sender_idx], c="blue")
plt.plot(t, S_t[:, sender_idx + 1], c="black")

We can also plot gene expression for all the cells on the lattice using functions from the `lsig.viz` module

In [ ]:
# Plot at days 1, 2, 3, 4
time_points_days = np.arange(1, 5)                     # units of days
time_points = time_points_days / lsig.t_to_units(1)    # simulation time units (dimensionless)

L = 20   
xlim = -L/2, L/2
ylim = -L/2, L/2

fig = plt.figure()
fig.set_facecolor("black")
for plot_idx, tp in enumerate(time_points):
    ax = fig.add_subplot(2, 2, plot_idx + 1)
    
    i = np.searchsorted(t, tp)
    lsig.viz.plot_hex_sheet(
        ax=ax,
        X=X_t[i],
        var=S_t[i],
        rho=rho_t[i],
        cmap=lsig.viz.kgy,
        title=fr"Day {i + 1:.1f}, $\rho={{{rho_t[i]:.2f}}}$",
        xlim=xlim,
        ylim=ylim,
        scalebar=True,
    )

We can also animate this and save it to a file in `lsig`'s default plotting directory (`lsig.plot_dir`)

In [ ]:
def animate_hex_sheet(
    fname,
    X_t,
    var_t,
    rho_t,
    title_func,
    cmap=lsig.viz.kgy,
    scalebar=True,
    n_frames=50,
    fps=10,
    dpi=120,
    save_dir=lsig.plot_dir,
    writer="ffmpeg",
    progress=True,
    **hex_kw
):
    
    from matplotlib import animation
    
    nt = X_t.shape[0]
    frames = lsig.vround(np.linspace(0, nt - 1, n_frames))

    fig, ax = plt.subplots()

    def anim(i):
        """Plot frame of animation"""
        ax.clear()
        lsig.viz.plot_hex_sheet(
            ax=ax, 
            X=X_t[frames[i]],
            var=var_t[frames[i]],
            rho=rho_t[frames[i]],
            title=title_func(frames, i),
            cmap=cmap,
            scalebar=scalebar,
            **hex_kw
        )

    _writer = animation.writers[writer](fps=fps, bitrate=1800)
    _anim_FA = animation.FuncAnimation(fig, anim, frames=n_frames, interval=200)

    # Get path and print to output
    fpath = save_dir.joinpath(fname).with_suffix(".mp4")
    print("Writing to:", fpath.resolve().absolute())
    
    # Save animation
    callback_func = lambda i, n: print(f"Frame {i+1} / {n}")
    _callback = dict(progress_callback=callback_func) if progress else {}
    _anim_FA.save(
        fpath,
        writer=_writer,
        dpi=dpi,
        **_callback
    )

Running the animation function might take a few minutes

In [ ]:
title_func = lambda frames, i: f"{t_days[frames[i]]:.2f} days"

animate_hex_sheet(
    "example.mp4",
    X_t=X_t,
    var_t=S_t,
    rho_t=rho_t,
    title_func=title_func,
    xlim=xlim,
    ylim=ylim,
);

In [ ]:
from IPython.display import Video

# Play video
Video(lsig.plot_dir.joinpath("example.mp4"))

We can change this parameter configuration and re-run the entire pipeline by passing parameter updates to the `ex` object

In [ ]:
# Make updates to parameters
config_updates = {
    "alpha": 2.0,    # Weaker promoter (default 3.0)
    "g": 0.5,        # Slower growth rate (1.0 is wild-type rate)
}

# Run with updates
run2 = ex.run(config_updates=config_updates)

# Extract data
run2_dir = Path(run2.observers[0].dir)
with h5py.File(run2_dir.joinpath("results.hdf5"), "r") as f:
    
    t = np.array(f["t"])
    t_days = lsig.t_to_units(t)
    
    rho_t = np.array(f["rho_t"])
    S_t = np.array(f["S_t"])
    R_t = np.array(f["R_t"])
    sender_idx = np.array(f["sender_idx"])
    
    X = np.array(f["X"])                    
    X = X - X[sender_idx]                   
    X_t = lsig.transform_lattice(X, rho_t)  

    
# Animate results
title_func = lambda frames, i: f"{t_days[frames[i]]:.2f} days"
animate_hex_sheet(
    "example_v2.mp4",
    X_t=X_t,
    var_t=S_t,
    rho_t=rho_t,
    title_func=title_func,
    xlim=xlim,
    ylim=ylim,
 )

In [ ]:
from IPython.display import Video

# Play video
Video(lsig.plot_dir.joinpath("example_v2.mp4"))